In [1]:
import getpass
conf = sc.getConf().setAppName("the {}\'s spark app".format(getpass.getuser()))
sc.stop()
sc = SparkContext(conf=conf)

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Spark DF practice').master('yarn').getOrCreate()

In [6]:
import pandas as pd
from pyspark.sql import *
from pyspark.sql import Window as W
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [3]:
spark

In [2]:
commits= spark.read.parquet("/data/clickhouse_2022_repodata_parquet_v2/commits").cache()

file_changes= spark.read.parquet("/data/clickhouse_2022_repodata_parquet_v2/file_changes").cache()

line_changes= spark.read.parquet("/data/clickhouse_2022_repodata_parquet_v2/line_changes").cache()

In [3]:
commits.printSchema()
file_changes.printSchema()
line_changes.printSchema()

root
 |-- hash: string (nullable = true)
 |-- author: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- message: string (nullable = true)
 |-- files_added: integer (nullable = true)
 |-- files_deleted: integer (nullable = true)
 |-- files_renamed: integer (nullable = true)
 |-- files_modified: integer (nullable = true)
 |-- lines_added: integer (nullable = true)
 |-- lines_deleted: integer (nullable = true)
 |-- hunks_added: integer (nullable = true)
 |-- hunks_removed: integer (nullable = true)
 |-- hunks_changed: integer (nullable = true)

root
 |-- change_type: string (nullable = true)
 |-- path: string (nullable = true)
 |-- old_path: string (nullable = true)
 |-- file_extension: string (nullable = true)
 |-- lines_added: integer (nullable = true)
 |-- lines_deleted: integer (nullable = true)
 |-- hunks_added: integer (nullable = true)
 |-- hunks_removed: integer (nullable = true)
 |-- hunks_changed: integer (nullable = true)
 |-- commit_hash: string (nullable = 

In [50]:
commits.limit(10).toPandas()

,hash,author,time,message,files_added,files_deleted,files_renamed,files_modified,lines_added,lines_deleted,hunks_added,hunks_removed,hunks_changed
0,4c266d1e5da622f249a4ff737aa95089fc54999a,Anton Popov,2020-07-29 17:09:38,fix wrong index analysis with functions,2,0,0,1,78,1,7,0,1
1,06f88a3caa4af03c65d6d0f263a168f5b9654995,root,2020-07-29 17:09:43,Updated formating of insert.py tests,0,0,0,2,65,44,19,1,23
2,2bdf276687a0b6769f330b993959966239c96936,Anton Popov,2020-07-29 17:20:17,fix access to redis dictionary after connectio...,0,0,0,1,6,0,2,0,0
3,65771ad516b775bef9b05f7b0ed56f2c186d6a23,Alexander Kuzmenkov,2020-07-29 18:05:38,Fix the AST fuzzer error message,0,0,0,1,15,10,2,1,4
4,a82ac93d245c8fed2952e4c5490ec226939f1c26,MyroTk,2020-07-29 18:16:06,Updated to include ipv6 hosts,0,0,0,1,1,1,0,0,1
5,1d6c2ba689563e692a0c6888a228b7fb7a7cefd0,Nikita Mikhaylov,2020-07-29 18:37:57,rewrite other test + remove useless includes,1,1,0,27,215,155,4,43,8
6,d6102869e5ce13ac563938d66fccb3a13e8ebccf,Azat Khuzhin,2020-07-29 19:00:46,Avoid re-loading completion from the history f...,0,0,0,1,14,1,1,0,1
7,b7eee3afe05145cacf4823487190fbffaa1e9050,Azat Khuzhin,2020-07-29 19:00:46,Add a comment of explicit flock() in ReplxxLin...,0,0,0,1,4,0,1,0,0
8,5a6a40437a33865c0af7237f5fb979b569f116f2,Nikita Mikhaylov,2020-07-29 19:14:48,remove includes from all Function sources,0,0,0,66,1,90,0,84,1
9,644d4c7aaa6c77432f436a1dab9cbe7b5aa2d1e0,Pavel Kovalenko,2020-07-29 19:15:20,Cache mark and index files for DiskS3.,0,0,0,5,92,50,4,4,7


In [49]:
file_changes.limit(10).toPandas()

,change_type,path,old_path,file_extension,lines_added,lines_deleted,hunks_added,hunks_removed,hunks_changed,commit_hash,commit_files_added,commit_files_deleted,commit_files_renamed,commit_files_modified,commit_lines_added,commit_lines_deleted,commit_hunks_added,commit_hunks_removed,commit_hunks_changed
0,Modify,dbms/src/Interpreters/JoinToSubqueryTransformV...,None,h,1,1,0,0,1,48ee3e83fad8bca2d2eda28297aadde2faccb94d,1,0,0,7,298,99,19,3,28
1,Modify,dbms/src/Interpreters/TranslateQualifiedNamesV...,None,cpp,38,11,4,0,3,48ee3e83fad8bca2d2eda28297aadde2faccb94d,1,0,0,7,298,99,19,3,28
2,Modify,dbms/src/Parsers/ASTAsterisk.h,None,h,8,0,2,0,0,48ee3e83fad8bca2d2eda28297aadde2faccb94d,1,0,0,7,298,99,19,3,28
3,Modify,dbms/src/Parsers/ASTQualifiedAsterisk.h,None,h,8,0,2,0,0,48ee3e83fad8bca2d2eda28297aadde2faccb94d,1,0,0,7,298,99,19,3,28
4,Modify,dbms/src/Storages/MergeTree/MergeTreeBloomFilt...,None,cpp,103,0,2,0,0,4343ede94426b84a00a3ec859b12321a8239978e,0,0,0,2,107,2,2,0,2
5,Modify,dbms/src/Storages/MergeTree/MergeTreeBloomFilt...,None,h,4,2,0,0,2,4343ede94426b84a00a3ec859b12321a8239978e,0,0,0,2,107,2,2,0,2
6,Modify,dbms/src/Interpreters/BloomFilter.cpp,None,cpp,5,0,1,0,0,f7c091d497bdc9537e5ce2a222197ba074b0d455,0,0,0,9,47,12,16,1,8
7,Modify,dbms/src/Interpreters/BloomFilter.h,None,h,1,0,1,0,0,f7c091d497bdc9537e5ce2a222197ba074b0d455,0,0,0,9,47,12,16,1,8
8,Modify,dbms/src/Storages/MergeTree/MergeTreeBloomFilt...,None,cpp,21,10,3,1,6,f7c091d497bdc9537e5ce2a222197ba074b0d455,0,0,0,9,47,12,16,1,8
9,Modify,dbms/src/Storages/MergeTree/MergeTreeBloomFilt...,None,h,2,1,1,0,1,f7c091d497bdc9537e5ce2a222197ba074b0d455,0,0,0,9,47,12,16,1,8


In [7]:
pd.set_option('display.max_columns', None)


In [47]:
line_changes.limit(10).toPandas()

,sign,line_number_old,line_number_new,hunk_num,hunk_start_line_number_old,hunk_start_line_number_new,hunk_lines_added,hunk_lines_deleted,hunk_context,line,indent,line_type,prev_commit_hash,prev_time,file_change_type,path,old_path,file_extension,file_lines_added,file_lines_deleted,file_hunks_added,file_hunks_removed,file_hunks_changed,commit_hash,commit_hunks_changed
0,-1,27,27,1,27,27,1,1,"$CLICKHOUSE_CLIENT --query=""SELECT * FROM set_...","$CLICKHOUSE_CLIENT --query=""DROP TABLE set_idx;""",255.0,Code,f1cc83ca9e9d76803480f6d2d158c5cffe98086d,2019-04-16 15:13:13,Modify,tests/queries/0_stateless/00907_set_index_max_...,None,sh,1.0,1.0,0.0,0.0,1.0,6adbd516cc16b5476d5e474a9000cbb6e870489b,1.0
1,1,28,27,1,27,27,1,1,"$CLICKHOUSE_CLIENT --query=""SELECT * FROM set_...","$CLICKHOUSE_CLIENT --query=""DROP TABLE set_idx;""",255.0,Code,f1cc83ca9e9d76803480f6d2d158c5cffe98086d,2019-04-16 15:13:13,Modify,tests/queries/0_stateless/00907_set_index_max_...,None,sh,1.0,1.0,0.0,0.0,1.0,6adbd516cc16b5476d5e474a9000cbb6e870489b,1.0
2,1,0,17,1,0,1,30,0,None,\t255\tPunct\t\t\t0000-00-00 00:00:00\tAdd\tte...,NaN,None,None,NaT,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN
3,-1,270,270,1,270,270,1,1,None,url = https://github.com/nats-io/nats.c.git,255.0,Code,944a729fc531f5bce1569c414fcc5546914d26bc,2022-05-12 11:58:49,Modify,.gitmodules,None,None,1.0,1.0,0.0,0.0,1.0,30ff0de33e2353af3057af62ade67b50ce290326,1.0
4,1,271,270,1,270,270,1,1,None,url = https://github.com/tchepavel/nats.c.git,255.0,Code,944a729fc531f5bce1569c414fcc5546914d26bc,2022-05-12 11:58:49,Modify,.gitmodules,None,None,1.0,1.0,0.0,0.0,1.0,30ff0de33e2353af3057af62ade67b50ce290326,1.0
5,-1,23,33,4,23,33,4,1,String NATSConnectionManager::connectionInfoFo...,return configuration.host + \':\' + toString(c...,255.0,Code,944a729fc531f5bce1569c414fcc5546914d26bc,2022-05-12 11:58:49,Modify,src/Storages/NATS/NATSConnection.cpp,None,cpp,36.0,38.0,4.0,1.0,11.0,f53ec6a3a5ba605d58a09912226ac9f889af7d83,57.0
6,1,24,33,4,23,33,4,1,String NATSConnectionManager::connectionInfoFo...,if (!configuration.url.empty()) {,255.0,Code,944a729fc531f5bce1569c414fcc5546914d26bc,2022-05-12 11:58:49,Modify,src/Storages/NATS/NATSConnection.cpp,None,cpp,36.0,38.0,4.0,1.0,11.0,f53ec6a3a5ba605d58a09912226ac9f889af7d83,57.0
7,-1,53,65,5,53,65,0,18,bool NATSConnectionManager::reconnect(),SubscriptionPtr NATSConnectionManager::createS...,255.0,Code,944a729fc531f5bce1569c414fcc5546914d26bc,2022-05-12 11:58:49,Modify,src/Storages/NATS/NATSConnection.cpp,None,cpp,36.0,38.0,4.0,1.0,11.0,f53ec6a3a5ba605d58a09912226ac9f889af7d83,57.0
8,-1,54,65,5,53,65,0,18,bool NATSConnectionManager::reconnect(),{,255.0,Punct,944a729fc531f5bce1569c414fcc5546914d26bc,2022-05-12 11:58:49,Modify,src/Storages/NATS/NATSConnection.cpp,None,cpp,36.0,38.0,4.0,1.0,11.0,f53ec6a3a5ba605d58a09912226ac9f889af7d83,57.0
9,-1,55,65,5,53,65,0,18,bool NATSConnectionManager::reconnect(),std::lock_guard lock(mutex);,255.0,Code,944a729fc531f5bce1569c414fcc5546914d26bc,2022-05-12 11:58:49,Modify,src/Storages/NATS/NATSConnection.cpp,None,cpp,36.0,38.0,4.0,1.0,11.0,f53ec6a3a5ba605d58a09912226ac9f889af7d83,57.0


### Посмотреть актиновность Миловидова

Испольвать сальтирование; Проблема перекоса данных data skew. При join по такому ключу спарк не может равномерно распределить нагрузку по воркерам - и один из них получает больше нагрузки чем другие


In [28]:
# 1. first use groupBy key
line_changes.groupBy(col("commit_hash").isNull()).count().show()

+---------------------+-------+
|(commit_hash IS NULL)|  count|
+---------------------+-------+
|                 true|   1173|
|                false|3706031|
+---------------------+-------+



In [29]:
dropna_line_changes = line_changes.filter(line_changes.commit_hash.isNotNull())

In [31]:
res = (dropna_line_changes
    .join(file_changes, dropna_line_changes.commit_hash == file_changes.commit_hash, "left")
    .join(commits, dropna_line_changes.commit_hash == commits.hash, "left")
    .select(commits.author, dropna_line_changes.file_lines_added, dropna_line_changes.file_lines_deleted))


In [34]:
res.groupBy(col("author")).count().orderBy("count", ascending=False).show(20)

+-------------------+--------+
|             author|   count|
+-------------------+--------+
|   Alexey Milovidov|77693987|
|Mikhail f. Shiryaev|41646478|
|              s-kat|27258099|
|            CurtizJ|20364564|
|            Mikhail|17428481|
|               Ivan|14114840|
|      Alexey Boykov|10692510|
|            proller| 8131804|
|   Nikolai Kochetov| 5264636|
|     Ivan Lezhankin| 3736818|
|             MyroTk| 3243188|
|              myrrc| 3104617|
|        Maksim Kita| 2917352|
|     Vitaly Baranov| 2494085|
|   alexey-milovidov| 2233864|
|       kreuzerkrieg| 1962173|
|Nikolay Degterinsky| 1859890|
|        Anton Popov| 1784461|
|       Azat Khuzhin| 1748650|
|             Li Yin| 1578577|
+-------------------+--------+
only showing top 20 rows



### Найти самые часто исправляемые файлы
и собрать массив кто их правил

In [37]:
most_modified_df = (dropna_line_changes
    .join(file_changes, dropna_line_changes.commit_hash == file_changes.commit_hash, "left")
    .join(commits, dropna_line_changes.commit_hash == commits.hash, "left")
    .select(file_changes.path, commits.author))

In [38]:
most_modified_df.groupBy(col('path')).count().orderBy("count", ascending=False).show(20)

+--------------------+------+
|                path| count|
+--------------------+------+
|src/Functions/CMa...|247317|
|  src/CMakeLists.txt|225001|
|         .gitmodules|211975|
|      CMakeLists.txt|209441|
|programs/server/S...|206424|
|src/Common/Freque...|187237|
|src/Functions/Fun...|183904|
|dbms/src/Storages...|182694|
|src/Functions/Fun...|182087|
|dbms/src/Storages...|179309|
|cmake/find/cld2.c...|177957|
|programs/server/e...|177955|
|programs/server/c...|177955|
|src/Storages/Stor...|168982|
|dbms/src/Interpre...|167363|
|src/Storages/Merg...|166921|
| src/Core/Settings.h|165679|
|src/Functions/reg...|154836|
|dbms/src/Interpre...|153122|
|src/Interpreters/...|152267|
+--------------------+------+
only showing top 20 rows



In [45]:

authors_by_path = (
    most_modified_df
    .groupBy("path")
    .agg(
        collect_set("author").alias("authors"),
        count("*").alias("modification_count")
    )
    .orderBy("modification_count", ascending=False)
)

authors_by_path.show(20)

+--------------------+--------------------+------------------+
|                path|             authors|modification_count|
+--------------------+--------------------+------------------+
|src/Functions/CMa...|[Andr0901, Boloni...|            247317|
|  src/CMakeLists.txt|[msaf1980, Mikhai...|            225001|
|         .gitmodules|[Ivan Blinkov, Al...|            211975|
|      CMakeLists.txt|[Ivan Blinkov, ch...|            209441|
|programs/server/S...|[Timur Magomedov,...|            206424|
|src/Common/Freque...|[Maksim Kita, s-k...|            187237|
|src/Functions/Fun...|[Li Yin, Azat Khu...|            183904|
|dbms/src/Storages...|[chertus, Vitaliy...|            182694|
|src/Functions/Fun...|[Li Yin, Maksim K...|            182087|
|dbms/src/Storages...|[Akazz, maiha, Vi...|            179309|
|cmake/find/cld2.c...|[s-kat, Nikolay D...|            177957|
|programs/server/e...|[s-kat, Nikolay D...|            177955|
|programs/server/c...|[s-kat, Nikolay D...|            